# Merging data

Data that is going to be used in model creation is currently spread across datasets:
- `listings_modified.csv` - main dataset, contains data about listings from cetris.ca;
- `extra_data_modified.csv` - extra data about listings from above dataset;
- `policeCoord.csv` & `firestations.csv` - locations of police stations and fire stations in Montréal
- `montreal_hpi.csv`
- `sociodemo_modified.csv` - Montréal sociodemographical data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Preparing listings data

In [2]:
listings = pd.read_csv('data/listings_modified.csv')

### 1.1. Dropping redundant columns

(see EDA: `eda_merged.ipynb`)

In [3]:
redundant_cols = [
    '__type', 'ShortCity',
    
    'RevenuEffectif', 'ShortDescription', 'Langue', 'SuperficieTerrain',
    'NbEquipements', 'NbAnimals', 'NbCultures', 'LocationPrice', 'LocationPriceDesc',
    
    'CatgCode', 'PhotoText', 'DiffusionWebUrl', 'PhotoUrl', 'PhotoPrintingUrl', 
    'FinancialToolUrl', 'PasserelleUrl', 'CommunityUrl', 'GoogleStreetViewUrl', 
    'WalkScoreUrl', 'ShareThisUrl', 'SummaryUrl', 'PrintingPageUrl', 
    'GoogleMapAddressLink', 
    
    'IsFavorite', 'IsRemoved', 'IsSubject', 'IsStrictQueryMatch', 'IsFavoriteDisplayed',
    'IsRemovedDisplayed', 'IsEmptyView', 'IsMissingView', 'IsInFavorite',

    'IsNew', 'IsNewPrice'
]
listings.drop(redundant_cols,
              axis=1,
              inplace=True)

### 1.2. Converting `VisiteVirtuelleUrl` & `OpenHouse` to boolean columns

In [4]:
listings['OpenHouse'] = ~listings['OpenHouse'].isna()

In [5]:
listings['VisiteVirtuelleUrl'] = ~listings['VisiteVirtuelleUrl'].isna()
listings.rename({'VisiteVirtuelleUrl': 'VirtualTour'}, axis=1, inplace=True)

### 1.3. `Utilisation`

Connect "Commercial and residential"  and "Residential and commercial" into one

In [6]:
listings['Utilisation'].value_counts()

Residential only                     2299
Commercial only                       392
Residential and commercial            255
Commercial and office space           139
Commercial and residential            124
Commercial and industrial              22
Industrial and offices                 19
Industrial only                        15
Other                                  14
Offices only                           14
Multi-family dwelling                   5
Retirement home                         4
Commercial or industrial (income)       4
Name: Utilisation, dtype: int64

In [7]:
listings.loc[
    listings['Utilisation'] == 'Residential and commercial', 
    'Utilisation'
] = 'Commercial and residential'

### 1.4. `BuyPriceDesc`

In [8]:
listings['BuyPriceDesc'].value_counts()

 /square foot    20
Name: BuyPriceDesc, dtype: int64

In [9]:
listings.loc[listings['BuyPriceDesc'].isna(), 'BuyPrice'].describe()

count    1.464700e+04
mean     5.204339e+05
std      5.965235e+05
min      9.480320e+03
25%      2.590000e+05
50%      3.790000e+05
75%      5.599000e+05
max      1.388000e+07
Name: BuyPrice, dtype: float64

In [10]:
listings.loc[listings['BuyPriceDesc'] == ' /square foot', 'BuyPrice'].describe()

count    2.000000e+01
mean     8.246817e+04
std      2.718385e+05
min      3.130000e+00
25%      6.725000e+01
50%      1.556900e+02
75%      1.580800e+02
max      1.126332e+06
Name: BuyPrice, dtype: float64

In [11]:
# wydaje mi sie ze te co mają tu NA to są pełne ceny, a te co nie mają to są ceny za ft2
# problem w tym ze chyba nie mamy metrażu do nich wiec nie bardzo będzie jak to przeliczyć
# (ramke z metrażem tam niżej dołączyłem)

### 1.5. Renaming columns with french names to english

In [12]:
listings.rename({
    'RevenuBrutPotentiel': 'GrossPotentialIncome',
    'NbPieces': 'NumberOfRooms',
    'NbChambres': 'NumberOfBedrooms',
    'NbSallesEaux': 'NumberOfBathroomsWithShower',
    'NbSallesBains': 'NumberOfBathroomsWithBathtub',
    'NbFoyerPoele': 'NumberOfFireplaces',
    'NbGarages': 'NumberOfGarages',
    'NbStationnements': 'NumberOfParkingLots',
    'NbPiscines': 'NumberOfPools',
    'NbBordEaux': 'NbBordEaux',
}, axis=1, inplace=True)

In [13]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14667 entries, 0 to 14666
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   MlsNumber                     14667 non-null  int64  
 1   CategoryCode                  14667 non-null  object 
 2   Lat                           14667 non-null  float64
 3   Lng                           14667 non-null  float64
 4   Address                       14667 non-null  object 
 5   City                          14667 non-null  object 
 6   Quartier                      11488 non-null  object 
 7   PostalCode                    14667 non-null  object 
 8   LongDescription               12922 non-null  object 
 9   BuyPrice                      14667 non-null  float64
 10  BuyPriceDesc                  20 non-null     object 
 11  Category                      14667 non-null  object 
 12  Construction                  14667 non-null  int64  
 13  O

### 1.6. Columns with location data

#### `Address`

In [14]:
listings['Address'].value_counts()

Boul. Gouin O.                       13
Rue Lakeview                          3
50e Avenue (R.-d.-P.)                 3
59e Avenue                            3
Boul. Beaconsfield                    3
                                     ..
1321, Rue Sherbrooke O., apt. B51     1
1625, Place Glengarry                 1
61, Rue Montigny                      1
9699 - 9701, Rue Jeannette            1
1118A, Av. Laurier E.                 1
Name: Address, Length: 14623, dtype: int64

`Address` column is quite specific and probably not that useful since there are `Lat` and `Lng` columns (?)

#### `City`

In [15]:
print(len(listings['City'].unique()))
listings['City'].value_counts()

33


Ville-Marie (Montréal)                                 2087
Mercier/Hochelaga-Maisonneuve (Montréal)               1124
Rivière-des-Prairies/Pointe-aux-Trembles (Montréal)     920
Le Plateau-Mont-Royal (Montréal)                        871
Le Sud-Ouest (Montréal)                                 816
Côte-des-Neiges/Notre-Dame-de-Grâce (Montréal)          736
Rosemont/La Petite-Patrie (Montréal)                    730
Villeray/Saint-Michel/Parc-Extension (Montréal)         729
Saint-Laurent (Montréal)                                704
Ahuntsic-Cartierville (Montréal)                        696
Verdun/Île-des-Soeurs (Montréal)                        677
Pierrefonds-Roxboro (Montréal)                          511
LaSalle (Montréal)                                      470
Montréal-Nord (Montréal)                                433
Lachine (Montréal)                                      361
Dollard-Des Ormeaux                                     333
Côte-Saint-Luc                          

This column will be useful later to merge this data frame with sociodemographical data.

Removing "(Montréal)" from the end of some records' values (also will be useful later):

In [16]:
listings['City'] = listings['City'].apply(
    lambda city: city[:-11] 
                 if city.endswith('(Montréal)') 
                 else city
)

#### `Quartier`

In [17]:
print(len(listings['Quartier'].unique()))
listings['Quartier'].value_counts()

52


Central                           1244
Central West                       831
Le Plateau-Mont-Royal              735
Mercier                            591
Rivière-des-Prairies               504
Hochelaga-Maisonneuve              499
Île-des-Soeurs                     442
Rosemont                           431
Saint-Henri/Petite Bourgogne       428
Pointe-aux-Trembles                416
Côte-des-Neiges                    400
Villeray                           342
East                               336
Notre-Dame-de-Grâce                336
Saint-Michel                       302
La Petite-Patrie                   299
Old Montréal                       251
Verdun                             235
Saint-Paul/Émard                   228
Nouveau Bordeaux                   192
Pierrefonds/Central East           176
L'Île-Bizard                       163
Pointe-Saint-Charles               160
West                               146
Ahuntsic West                      143
Mile-End                 

#### `PostalCode`

In [18]:
print(len(listings['PostalCode'].unique()))
listings['PostalCode'].value_counts()

7996


H3B 3C3    61
H3C 1W4    57
H3G 2M5    52
H3G 2H5    45
H1V 2L1    45
           ..
H1W 1M3     1
H4K 2E6     1
H3Z 1M7     1
H2A 2K4     1
H2P 2J3     1
Name: PostalCode, Length: 7996, dtype: int64

In [19]:
# nie wiem, wywalamy te dwie ostatnie kolumny i Address jak i tak mamy lat i lng czy jakoś je kodujemy czy coś?

### 1.7. Other columns

In [20]:
listings['LongDescription'].head(n=10)

0    PRIX POUR VENTE RAPIDE. Superbe condo ensoleil...
1    Duplex à vendre à Ahuntsic près de la rivière,...
2    rare sur le marché! ideal pour dentiste ou med...
3    Excellent opportunity for builder/ contractor....
4    Beautiful condo built in 2011 still under guar...
5    One bedroom condo in a sought out area of Ahun...
6    Grand duplex avec garage dans un secteur très ...
7    Hard to imagine a more inspiring space than th...
8    You want to become a Florist  !!! So this busi...
9    Great Deal !!! . 4plex for sale in the heart o...
Name: LongDescription, dtype: object

In [21]:
listings['Category'].value_counts()

Condo/Loft, Apartment, Detached                       3338
Condo/Loft, Apartment, Attached                       2255
Condo/Loft, Apartment, Semi-detached                  1385
Single Family, Two or more storey, Detached           1344
Single Family, Bungalow, Detached                      647
                                                      ... 
Revenue Property, Detached, Shopping Center              1
Commercial, Florist                                      1
Revenue Property, Attached, 8-Plex                       1
Revenue Property, Detached, 42 units                     1
Revenue Property, Detached, 28 units, 2 businesses       1
Name: Category, Length: 408, dtype: int64

In [22]:
listings['CategoryCode'].value_counts()

COP    7698
UNI    3522
PPR    2587
PCI     719
TER     140
FER       1
Name: CategoryCode, dtype: int64

In [23]:
all_categories = set()
listings['Category'].apply(lambda s: all_categories.update(s.split(', ')))
len(all_categories)

331

In [24]:
listings['PageTitle'].value_counts()

Condos for sale in Mercier/Hochelaga-Maisonneuve (Montréal), Montréal (Island) – 2540, Av. Bilaudeau, 17659544 | Centris                                  1
Condos for sale in Montréal-Nord (Montréal), Montréal (Island) – 3681, Boul. Henri-Bourassa E., apt. 302, 15086430 | Centris                              1
Condos for sale in Ville-Marie (Montréal), Montréal (Island) – 2020, Rue Falardeau, apt. 101, 28792616 | Centris                                          1
Houses for sale in Lachine (Montréal), Montréal (Island) – 325, Tsse Louis-Basile-Pigeon, 28901276 | Centris                                              1
Commercial building for sale in Verdun/Île-des-Soeurs (Montréal), Montréal (Island) – 3135, Rue Wellington, 28572456 | Centris                            1
                                                                                                                                                         ..
Condos for sale in Ville-Marie (Montréal), Montréal (Island) – 6

In [25]:
# nie wiem co z tymi zrobic, intuicja mi mowi zeby to wywalic ale nwm
# mozna niby Category jakos zakodwac
# ale z drugiej strony mamy CategoryCode wiec nie wiem czy to jest konieczne

## 2. Merging data frames

### 2.1. `extra_data_modified.csv`

In [26]:
extra_data = pd.read_csv('data/extra_data_modified.csv')

In [27]:
extra_data.columns

Index(['MlsNumber', 'LivingArea', ',Bibliotheque',
       '.picerie distance de marche', '3min from Verdun metro', 'AIRPORT',
       'ANGRIGNON Metro', 'AR�NA+PISCINES PUBLIC INTERIEUR',
       'AVH German school ', 'Access to highway 20',
       ...
       '�cole de Musique', '�cole des Adultes.', '�cole primaire',
       '�cole secondaire', '�coles primaires', '�picerie',
       '�picerie Loblaws Angus', '�picerie M�tro', '�piceries',
       '�piceries fines services'],
      dtype='object', length=591)

In [28]:
final_df = pd.merge(listings, extra_data, on='MlsNumber', how='left')

In [29]:
final_df.loc[final_df['BuyPriceDesc'] == ' /square foot', 'LivingArea']

505     NaN
519     NaN
714     NaN
2717    NaN
5583    NaN
6398    NaN
6534    NaN
6739    NaN
9158    NaN
10237   NaN
10251   NaN
10279   NaN
10365   NaN
10368   NaN
10381   NaN
10402   NaN
10596   NaN
10617   NaN
10634   NaN
14065   NaN
Name: LivingArea, dtype: float64

In [30]:
# wszystkie te co mają podaną cene w /ft2 nie mają podanej powierzchni :/

### 2.2. `policeCoord.csv` & `firestations.csv`

In [31]:
police_coord = pd.read_csv('data/policeCoord.csv').rename({'long': 'lng'}, axis=1)
firestations_coord = pd.read_csv('data/firestations.csv')

In [32]:
police_coord.head()

,lng,lat
0,-73.542094,45.583362
1,-73.617431,45.577735
2,-73.545913,45.556857
3,-73.701026,45.528533
4,-73.593463,45.585063


In [33]:
firestations_coord.head()

,address,lat,lng
0,550 Blvd. St. Pierre,45.502644,-73.559248
1,230 Blvd.Chèvremont,45.493473,-73.876561
2,13795 Blvd. Pierrefonds,45.493300,-73.849787
3,13 rue Centre Commercial,45.509543,-73.806234
4,10 Sunnydale,45.493874,-73.806822


Adding distances to the closest fire station and police station to the merged data frame:

In [34]:
def get_distance_to_closest_building(lat: float, lng: float, 
                                     buildings: pd.DataFrame) -> float:
    """
    Returns distance (in kilometres) from the building at latitude lat and
    longitude lng to the closest building from data frame passed as 
    buildings parameter
    """
    buildings['distance'] = buildings.apply(
        lambda row: np.sqrt(
            (row['lat'] - lat)**2 + 
            (row['lng'] - lng)**2), 
        axis=1)
    return np.min(buildings['distance']) * 111  # *111 to convert degrees to km

In [35]:
final_df['DistanceToFireStation'] = final_df.apply(
    lambda row: get_distance_to_closest_building(
        row['Lat'], row['Lng'], firestations_coord),
    axis=1)
final_df['DistanceToPoliceStation'] = final_df.apply(
    lambda row: get_distance_to_closest_building(
        row['Lat'], row['Lng'], police_coord),
    axis=1)

### 2.3. `montreal_hpi`

In [36]:
montreal_hpi = pd.read_csv('data/montreal_hpi.csv')

All columns in this data frame have extremely strong correlation with each other

In [37]:
montreal_hpi.head()

,Date,Composite_HPI,Single_Family_HPI,One_Storey_HPI,Two_Storey_HPI,Townhouse_HPI,Apartment_HPI,Composite_Benchmark,Single_Family_Benchmark,One_Storey_Benchmark,Two_Storey_Benchmark,Townhouse_Benchmark,Apartment_Benchmark
0,Jan 2005,100.0,100.0,100.0,100.0,100.0,100.0,192200,200200,165400,245400,178600,163300
1,Feb 2005,101.3,101.3,101.6,100.9,104.8,100.9,194700,202800,168100,247600,187200,164800
2,Mar 2005,102.4,102.6,103.1,102.0,104.6,101.4,196800,205400,170600,250300,186800,165600
3,Apr 2005,103.2,103.4,104.1,102.5,105.7,102.1,198400,207000,172200,251500,188800,166800
4,May 2005,103.6,103.9,104.7,102.8,104.9,102.4,199100,208000,173200,252300,187400,167300


In [38]:
# nie bardzo wiem jak to połączyć z resztą

### 2.4. `sociodemo_modified.csv`

In [39]:
sociodemo = pd.read_csv('data/sociodemo_modified.csv')

In [40]:
sociodemo.head()

,Municipality,Population,Variation,Density,avgAge,below15,below24,below44,below64,below65,...,University,College,Secondary,Apprentice,No,NonImmigrant,Immigrant,french,English,Others
0,Ahuntsic-Cartierville,126891,2,5256,41,16,11,29,26,18,...,36,16,20,10,18,60,40,64,10,26
1,Anjou,41928,3,3064,43,16,10,24,28,22,...,24,15,23,15,23,71,29,76,8,16
2,Baie-d'Urfé,3850,-1,639,43,18,14,16,32,20,...,57,13,18,3,9,70,30,19,72,9
3,Beaconsfield,19505,2,1772,40,19,15,18,32,16,...,51,18,18,5,8,76,24,21,73,6
4,Côte-des-Neiges/Notre-Dame-de-Grâce (Montréal),165031,0,7699,38,16,14,33,23,14,...,48,15,20,6,11,47,53,32,39,29


Reducing number of columns:

In [41]:
# to niżej skopiowałem z edy z tym ze zmieniłem troche zeby zostawic kolumne Municipality bo będzie potrzebna przy merge'owaniu

In [42]:
# Get correlation matrix 
corr = sociodemo.corr()

# Create a mask for values above 90% 
# But also below 100% since it variables correlated with the same one
mask = (sociodemo.corr() > 0.9) & (sociodemo.corr() < 1.0)
high_corr = corr[mask]

# Create a new column mask using any() and ~
col_to_filter_out = ~high_corr[mask].any()

# Leave Municipality column
cols_to_leave = list(high_corr.columns[col_to_filter_out])
cols_to_leave.insert(0, 'Municipality')

# Apply new mask
sociodemo = sociodemo[cols_to_leave]

Removing "(Montréal)" from the end of `Municipality` values:

In [43]:
sociodemo['Municipality'] = sociodemo['Municipality'].apply(
    lambda city: city[:-11] 
                 if city.endswith('(Montréal)') 
                 else city
)

In [44]:
np.sort(sociodemo['Municipality'].unique())

array(['Ahuntsic-Cartierville', 'Anjou', "Baie-d'Urfé", 'Beaconsfield',
       'Côte-Saint-Luc', 'Côte-des-Neiges/Notre-Dame-de-Grâce',
       'Dollard-Des Ormeaux', 'Dorval', 'Hampstead', 'Kirkland',
       "L'Île-Bizard/Sainte-Geneviève", 'LaSalle', 'Lachine',
       'Le Plateau-Mont-Royal', 'Le Sud-Ouest',
       'Mercier/Hochelaga-Maisonneuve', 'Mont-Royal', 'Montréal-Est',
       'Montréal-Nord', 'Montréal-Ouest', 'Outremont',
       'Pierrefonds-Roxboro', 'Pointe-Claire',
       'Rivière-des-Prairies/Pointe-aux-Trembles',
       'Rosemont/La Petite-Patrie', 'Saint-Laurent', 'Saint-Léonard',
       'Sainte-Anne-de-Bellevue', 'Senneville', 'Verdun/Île-des-Soeurs',
       'Ville-Marie', 'Villeray/Saint-Michel/Parc-Extension', 'Westmount'],
      dtype=object)

In [45]:
np.sort(final_df['City'].unique())

array(['Ahuntsic-Cartierville', 'Anjou', "Baie-d'Urfé", 'Beaconsfield',
       'Côte-Saint-Luc', 'Côte-des-Neiges/Notre-Dame-de-Grâce',
       'Dollard-Des Ormeaux', 'Dorval', 'Hampstead', 'Kirkland',
       "L'Île-Bizard/Sainte-Geneviève", 'LaSalle', 'Lachine',
       'Le Plateau-Mont-Royal', 'Le Sud-Ouest',
       'Mercier/Hochelaga-Maisonneuve', 'Mont-Royal', 'Montréal-Est',
       'Montréal-Nord', 'Montréal-Ouest', 'Outremont',
       'Pierrefonds-Roxboro', 'Pointe-Claire',
       'Rivière-des-Prairies/Pointe-aux-Trembles',
       'Rosemont/La Petite-Patrie', 'Saint-Laurent', 'Saint-Léonard',
       'Sainte-Anne-de-Bellevue', 'Senneville', 'Verdun/Île-des-Soeurs',
       'Ville-Marie', 'Villeray/Saint-Michel/Parc-Extension', 'Westmount'],
      dtype=object)

Merging both data frames:

In [46]:
final_df = pd.merge(final_df, sociodemo, left_on='City', right_on='Municipality')

## 3. Exporting final data frame

In [47]:
# ogólnie to ma bardzo duzo kolumn teraz, pochodzą one chyba w więszkości z extra_data
# nie wiem w sumie czy to dobrze czy źle ale na pewno warto to mieć na uwadze
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14672 entries, 0 to 14671
Columns: 658 entries, MlsNumber to Others
dtypes: bool(2), float64(595), int64(49), object(12)
memory usage: 73.6+ MB


In [48]:
# na razie zakomentowałem to bo na razie jeszcze ten notebook jest niedokończony 
# final_df.to_csv('full_data.csv', encoding='utf-8')